In [50]:
!pip list --format=freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20220215.212715


In [60]:
import pickle
import pandas as pd


In [61]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [62]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [65]:
y_pred.mean()

16.191691679979066

In [ ]:
year = 2021
month = 2

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [68]:
results = {'ride_id': df['ride_id'] , 'predictions': y_pred}

In [69]:
df_result = pd.DataFrame(results)

In [70]:
df_result.to_parquet(
    '../output/df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)